In [ ]:
import openai
import json
import re
import pandas as pd
import requests
import time
from google.colab import userdata

df = pd.read_csv("/content/part_1_mathdial.csv")

columns = [
    'question',
    'ground_truth',
    'student_incorrect_solution',
    'student_profile',
    'conversation',
    'state',
    'action_text',
    'done'
]

new_df = df[columns]

state_df = new_df.copy()
state_df['full_state'] = None

for i in range(0, len(new_df)):
  if 'START' in str(new_df.iloc[i]['state']):
    state_df.at[i, 'full_state'] = ['', 'START']
    continue

  prev_action_text = new_df.iloc[i-1]['action_text']
  curr_state = new_df.iloc[i]['state']

  state_df.at[i, 'full_state'] = [prev_action_text, curr_state]

state_df

state_df['conversation']
curr_pos = 1
if len(state_df) > 0:
  state_df.loc[0, 'convo_turn'] = curr_pos

for i in range(1, len(state_df)):
    # Check if a new conversation starts (0 follows a 1)
    if state_df.loc[i-1, 'done'] == 1 and state_df.loc[i, 'done'] == 0:
        # Reset position counter for new conversation
        curr_pos = 1
    else:
        # Increment position within the same conversation
        curr_pos += 1

    # Assign the current position within conversation
    state_df.loc[i, 'convo_turn'] = curr_pos * 1

state_df['convo_turn'] = state_df['convo_turn'].astype(int)
state_df['old_state'] = state_df['state']
state_df['state'] = state_df['full_state']
state_df

,question,ground_truth,student_incorrect_solution,student_profile,conversation,state,action_text,done,full_state,convo_turn,old_state
0,Nancy is filling an aquarium for her fish. She...,First calculate the volume of the aquarium by ...,The aquarium has a volume of 4 x 6 x 3 = 72 cu...,Steven is a 7th grade student. He has difficul...,"Teacher: (probing)Steven, If you had 4 of some...","[, START]","Teacher: (probing)Steven, If you had 4 of some...",0,"[, START]",1,START
1,Nancy is filling an aquarium for her fish. She...,First calculate the volume of the aquarium by ...,The aquarium has a volume of 4 x 6 x 3 = 72 cu...,Steven is a 7th grade student. He has difficul...,"Teacher: (probing)Steven, If you had 4 of some...","[Teacher: (probing)Steven, If you had 4 of som...",Teacher: (probing)So if Nancy triples the 18 c...,0,"[Teacher: (probing)Steven, If you had 4 of som...",2,Steven: I would have 12 of something.
2,Nancy is filling an aquarium for her fish. She...,First calculate the volume of the aquarium by ...,The aquarium has a volume of 4 x 6 x 3 = 72 cu...,Steven is a 7th grade student. He has difficul...,"Teacher: (probing)Steven, If you had 4 of some...",[Teacher: (probing)So if Nancy triples the 18 ...,Teacher: (generic)Exactly correct!,1,[Teacher: (probing)So if Nancy triples the 18 ...,3,Steven: She would have 54 cubic feet of water.
3,John is very unfit and decides to work up to d...,He needs to do 15*3=45 progressions\nThat will...,"To get to 15 reps, John will take 15 - 1 = 14 ...",Stephanie is a 7th grade student. She has diff...,"Teacher: (probing)Stephanie, How many days wil...","[, START]","Teacher: (probing)Stephanie, How many days wil...",0,"[, START]",1,START
4,John is very unfit and decides to work up to d...,He needs to do 15*3=45 progressions\nThat will...,"To get to 15 reps, John will take 15 - 1 = 14 ...",Stephanie is a 7th grade student. She has diff...,"Teacher: (probing)Stephanie, How many days wil...","[Teacher: (probing)Stephanie, How many days wi...",Teacher: (probing)So how many days will it tak...,0,"[Teacher: (probing)Stephanie, How many days wi...",2,"Student: It will take one day to do one step, ..."
...,...,...,...,...,...,...,...,...,...,...,...
7388,The time Juan takes to grab his lunch from his...,If Juan takes 4 hours to move from his office ...,Let's call the time Juan takes to read the boo...,Ayisha is a 7th grade student. She has problem...,Teacher: (probing)Hi Ayisha! Did you notice th...,[Teacher: (focus)Eight is half of what? That's...,"Teacher: (focus)How many pages is in the book,...",0,[Teacher: (focus)Eight is half of what? That's...,6,Student: Eight is half of sixteen. So if it ta...
7389,The time Juan takes to grab his lunch from his...,If Juan takes 4 hours to move from his office ...,Let's call the time Juan takes to read the boo...,Ayisha is a 7th grade student. She has problem...,Teacher: (probing)Hi Ayisha! Did you notice th...,[Teacher: (focus)How many pages is in the book...,Teacher: (probing)You have too much informatio...,0,[Teacher: (focus)How many pages is in the book...,7,"Student: The book has 4000 pages, and it takes..."
7390,The time Juan takes to grab his lunch from his...,If Juan takes 4 hours to move from his office ...,Let's call the time Juan takes to read the boo...,Ayisha is a 7th grade student. She has problem...,Teacher: (probing)Hi Ayisha! Did you notice th...,[Teacher: (probing)You have too much informati...,"Teacher: (focus)Leave the 8 hours out of it, t...",0,[Teacher: (probing)You have too much informati...,8,"Student: Yes, I understand. The book has 4000 ..."
7391,The time Juan takes to grab his lunch from his...,If Juan takes 4 hours to move from his office ...,Let's call the time Juan takes to read the boo...,Ayisha is a 7th grade student. She has problem...,Teacher: (probing)Hi Ayisha! Did you notice th...,"[Teacher: (focus)Leave the 8 hours out of it, ...","Teacher: (telling)Sorry Ayisha, you've gone a ...",0,"[Teacher: (focus)Leave the 8 hour

In [ ]:
def extract_json(text):
    """
    Extract JSON object from text that might contain additional content,
    then validate whether it fits the expected schema.

    Returns:
        dict or None: Extracted JSON object if valid, otherwise None
    """
    print(f"Raw text (last 200 chars): {text[-200:] if len(text) > 200 else text}")

    # 1) Direct JSON loading attempt
    try:
        data = json.loads(text)
        if validate_schema(data):
            return data
    except json.JSONDecodeError:
        pass

    # 2) Regex pattern (brute-force) for JSON blocks
    json_pattern = r'\s*(\{[^{}]*(?:\{[^{}]*\}[^{}]*)*\})\s*'
    matches = re.findall(json_pattern, text, re.DOTALL)
    if matches:
        # Try from the last match backward (often the last is the full JSON)
        for potential_json in reversed(matches):
            try:
                data = json.loads(potential_json)
                if validate_schema(data):
                    return data
            except json.JSONDecodeError:
                continue

    # 3) Aggressive brace matching from the last '{' found
    try:
        start_idx = text.rfind('{')
        if start_idx != -1:
            brace_count = 1
            i = start_idx + 1
            while i < len(text) and brace_count > 0:
                if text[i] == '{':
                    brace_count += 1
                elif text[i] == '}':
                    brace_count -= 1
                i += 1

            if brace_count == 0:
                json_str = text[start_idx:i]
                data = json.loads(json_str)
                if validate_schema(data):
                    return data
    except Exception as e:
        print(f"Error in aggressive JSON extraction: {str(e)}")

    # 4) Fallback pattern with your required fields (if you want to be very specific)
    # Adjusted to include problem_difficulty
    pattern = (
        r'\{\s*"state":\s*\{[^}]*\},\s*"action":\s*\{[^}]*\}\s*\}'
    )
    matches = re.findall(pattern, text, re.DOTALL)
    if matches:
        for potential_json in matches:
            try:
                data = json.loads(potential_json.strip())
                if validate_schema(data):
                    return data
            except json.JSONDecodeError:
                continue

    print("All extraction methods failed or schema validation failed. Returning None.")
    return None


def validate_schema(data):
    """
    Validate the JSON structure to ensure it contains the right keys and
    that numeric fields are truly numeric and in the right range, etc.
    """
    # Top-level keys we expect
    if not isinstance(data, dict):
        return False

    if "state" not in data or "action" not in data:
        return False

    # Validate 'state' keys
    state = data["state"]
    required_state_keys = [
        "misconception_type", "listen_to_feedback", "problem_progress",
        "problem_concept", "conversation_turn", "correct_solution",
        "problem_difficulty"
    ]
    for key in required_state_keys:
        if key not in state:
            return False

    # Check numeric fields
    if not isinstance(state["listen_to_feedback"], int):
        return False
    if state["listen_to_feedback"] not in (0, 1):
        return False

    if not isinstance(state["problem_progress"], int):
        return False
    if not (0 <= state["problem_progress"] <= 100):
        return False

    if not isinstance(state["conversation_turn"], int):
        return False

    if not isinstance(state["correct_solution"], int):
        return False
    if state["correct_solution"] not in (0, 1):
        return False

    # Check problem_difficulty
    if not isinstance(state["problem_difficulty"], int):
        return False
    if not (1 <= state["problem_difficulty"] <= 5):
        return False

    # Validate 'action' keys
    action = data["action"]
    required_action_keys = [
        "next_action_hint_strength", "next_action_category", "next_action_subcategory"
    ]
    for key in required_action_keys:
        if key not in action:
            return False

    # Check action numeric fields
    if not isinstance(action["next_action_hint_strength"], int):
        return False
    # Change this line in validate_schema()
    if not (0 <= action["next_action_hint_strength"] <= 4):
        return False

    # Check action category
    valid_action_categories = ["Focus", "Probing", "Telling", "Generic"]
    if action["next_action_category"] not in valid_action_categories:
        return False

    # next_action_subcategory: up to you how strictly to validate
    # For example, if Focus: could be one of [Seek Next Step, Confirm Calculation, etc.]
    # We'll do minimal checking here
    if not isinstance(action["next_action_subcategory"], str):
        return False

    return True


# Example script usage:

# Set your OpenAI API key
api_key = userdata.get("OPENAI_API_KEY")
if not api_key:
    raise ValueError("No API key found. Please set OPENAI_API_KEY in userdata.")

processed = 0
successful = 0
failed = 0

requests_per_minute = 20
delay_between_requests = 60 / requests_per_minute

for i, row in state_df.iterrows():
    # Rate limit
    if i > 0:
        time.sleep(delay_between_requests)

    # Gather your fields
    question = row.get("question", "")
    correct_solution = row.get("ground_truth", "")
    student_profile = row.get("student_profile", "")
    current_conversation_state = row.get("state", "")
    conversation_turn = row.get("convo_turn", "")
    action_text = row.get("action_text", "")

    # Updated prompt with problem_difficulty added
    prompt = f"""
You are analyzing math tutoring sessions to extract features for an AI tutor.

CONTEXT:
- Question: {question}
- Correct solution: {correct_solution}
- Student profile: {student_profile}
- Current conversation: {current_conversation_state}
- Conversation turn: {conversation_turn}
- Next action text: {action_text}

TASK:
Extract the following features from this tutoring interaction:

1. MISCONCEPTION TYPE - This classification MUST be determined SOLELY from the student profile, not from the conversation. It represents the student's core misunderstanding and should be CONSISTENT across all interactions with this same student. Choose ONE of these types based only on the student profile description:
- operation_error: Student applies the wrong operation (e.g., adding instead of multiplying)
- number_fact_error: Student makes errors in basic number facts (e.g., multiplication tables)
- variable_meaning_error: Student misunderstands what variables represent
- conceptual_error: Student has fundamental misconception about a math concept
- calculation_error: Student makes arithmetic errors during calculation
- problem_interpretation_error: Student misunderstands what the problem is asking

2. LISTEN TO FEEDBACK:
   - 1 if the student has incorporated the teacher's feedback in the current conversation state
   - 0 if not

3. PROBLEM PROGRESS - Numerical estimate (0-100) of how close, given the student's current conversation state, the student is to the correct solution. For example:
  - 0: No progress/completely wrong approach
  - 25: Beginning to understand but major errors remain
  - 50: Halfway to solution with some key insights
  - 75: Minor errors but mostly correct approach
  - 100: Complete and correct solution
  (You may use any value between 0-100, not just the examples above)

4. PROBLEM CONCEPT:
   The specific math domain/concept being tested. Based on the question, choose the one most relevant concept: "basic_arithmetic", "fractions_decimals_and_percents", "ratios_proportions_and_rates", "measurement_and_geometry", "counting_and_combinatorics", "money_and_finance", or "algebraic_relationships".

5. CONVERSATION TURN:
   The number of exchanges so far.

6. CORRECT SOLUTION:
   - 0 if the student's final answer is incorrect
   - 1 if correct

7. PROBLEM DIFFICULTY (1-3):
   Rate how difficult the math question is on a scale from 1 to 5, where:
   - 1 = Very easy, basic concepts, straightforward approach
   - 2 = Moderate, requires some thought, multiple steps
   - 3 = Challenging, complex approach, requires deeper understanding

8. NEXT ACTION HINT STRENGTH (0-4):
   How strong is the teacher's hint? (0=very minimal hint, 4=essentially giving away the entire solution)

9. NEXT ACTION CATEGORY:
   - Focus
   - Probing
   - Telling
   - Generic

10. NEXT ACTION SUBCATEGORY:
    Choose **exactly one** of the following, depending on the category:

    **Focus**:
    - "Seek Next Step"
    - "Confirm Calculation"
    - "Re-direct to Sub-Problem"
    - "Highlight Missing Info"

    **Probing**:
    - "Ask for Explanation"
    - "Seek Self-Correction"
    - "Hypothetical Variation"
    - "Check Understanding/Concept"
    - "Encourage Comparison"

    **Telling**:
    - "Partial Reveal (Strategy)"
    - "Full Reveal (Answer)"
    - "Corrective Explanation"

    **Generic**:
    - "Greeting/Farewell"
    - "Acknowledgment/Praise"
    - "Summarize Progress"
    - "General Inquiry/Filler"

FORMAT YOUR RESPONSE AS JSON:
{{
  "state": {{
    "misconception_type": "one_of_the_six_types",
    "listen_to_feedback": 0 or 1,
    "problem_progress": integer_between_0_and_100,
    "problem_concept": "specific_math_concept",
    "conversation_turn": integer,
    "correct_solution": 0 or 1,
    "problem_difficulty": integer_between_1_and_3
  }},
  "action": {{
    "next_action_hint_strength": integer_between_0_and_4,
    "next_action_category": "Focus/Probing/Telling/Generic",
    "next_action_subcategory": "pick_the_exact_one_subcategory_name"
  }}
}}

IMPORTANT:
- Return ONLY the JSON object with no additional explanation
- All numeric fields must be valid integers
"""

    headers = {
        "Content-Type": "application/json",
        "Authorization": f"Bearer {api_key}"
    }

    # Example payload structure; adjust to your actual usage:
    payload = {
        "model": "gpt-4-0613",
        "messages": [
            {"role": "user", "content": prompt.strip()}
        ],
        "max_tokens": 400
    }

    processed += 1

    try:
        print(f"Processing row {i} ...")
        response = requests.post("https://api.openai.com/v1/chat/completions", headers=headers, json=payload)

        # If rate limit error
        if response.status_code == 429:
            print("Rate limit exceeded. Waiting 60 seconds...")
            time.sleep(60)
            response = requests.post("https://api.openai.com/v1/chat/completions", headers=headers, json=payload)

        response.raise_for_status()  # Raise if not 2xx

        response_json = response.json()
        raw_response = response_json["choices"][0]["message"]["content"]

        # Store raw response
        state_df.loc[i, "raw_model_response"] = raw_response

        data = extract_json(raw_response)
        if data is not None:
            # Validate & store relevant fields
            # Because we do more thorough checks in validate_schema,
            # we can now simply parse them into your df if you like.

            # Flatten 'state' ...
            state_part = data.get("state", {})
            state_df.loc[i, "misconception_type"] = state_part.get("misconception_type", "")
            state_df.loc[i, "listen_to_feedback"] = state_part.get("listen_to_feedback", 0)
            state_df.loc[i, "problem_progress"] = state_part.get("problem_progress", 0)
            state_df.loc[i, "problem_concept"] = state_part.get("problem_concept", "")
            state_df.loc[i, "conversation_turn"] = state_part.get("conversation_turn", 0)
            state_df.loc[i, "correct_solution"] = state_part.get("correct_solution", 0)
            state_df.loc[i, "problem_difficulty"] = state_part.get("problem_difficulty", 3)  # Default to middle value
            # state_df.loc[i, "reasoning"] = state_part.get("reasoning", "")

            # Flatten 'action' ...
            action_part = data.get("action", {})
            state_df.loc[i, "next_action_hint_strength"] = action_part.get("next_action_hint_strength", 1)
            state_df.loc[i, "next_action_category"] = action_part.get("next_action_category", "")
            state_df.loc[i, "next_action_subcategory"] = action_part.get("next_action_subcategory", "")

            print(f"Successfully extracted JSON for row {i}")
            successful += 1
        else:
            state_df.loc[i, "extraction_failed"] = True
            print(f"Row {i}: Failed to extract valid JSON.")
            failed += 1

        # Save partial progress every 50 rows
        if i % 50 == 0:
            state_df.to_csv("part1_in_progress.csv", index=False)
            print(f"Progress saved at row {i}.")

    except Exception as e:
        state_df.loc[i, "error_message"] = str(e)
        state_df.loc[i, "extraction_failed"] = True
        print(f"Error processing row {i}: {str(e)}")
        failed += 1

    print("--------------------------------------------------------")

# Final save
state_df.to_csv("part1_mathdial_complete.csv", index=False)

print(f"\nProcessing complete:")
print(f"Total rows processed: {processed}")
print(f"Successful extractions: {successful}")
print(f"Failed extractions: {failed}")
if processed > 0:
    print(f"Success rate: {successful/processed*100:.2f}%")
else:
    print("No rows processed.")

Streaming output truncated to the last 5000 lines.
    "next_action_hint_strength": 3,
    "next_action_category": "Telling",
    "next_action_subcategory": "Corrective Explanation"
  }
}
Successfully extracted JSON for row 104
--------------------------------------------------------
Processing row 105 ...
Raw text (last 200 chars): rrect_solution": 0,
    "problem_difficulty": 2
  },
  "action": {
    "next_action_hint_strength": 2,
    "next_action_category": "Probing",
    "next_action_subcategory": "Ask for Explanation"
  }
}
Successfully extracted JSON for row 105
--------------------------------------------------------
Processing row 106 ...
Raw text (last 200 chars): versation_turn": 8, "correct_solution": 0, "problem_difficulty": 2}, "action": {"next_action_hint_strength": 3, "next_action_category": "Focus", "next_action_subcategory": "Re-direct to Sub-Problem"}}
Successfully extracted JSON for row 106
--------------------------------------------------------
Processing row 107 

KeyboardInterrupt: 

In [52]:
import pandas as pd

# Predefined mapping dictionary (1-based hint strengths).
ACTION_MAP = {
  "Focus": {
    "Seek Next Step": {
      "1": 0,
      "2": 1,
      "3": 2,
      "4": 3,
      "5": 4
    },
    "Confirm Calculation": {
      "1": 5,
      "2": 6,
      "3": 7,
      "4": 8,
      "5": 9
    },
    "Re-direct to Sub-Problem": {
      "1": 10,
      "2": 11,
      "3": 12,
      "4": 13,
      "5": 14
    },
    "Highlight Missing Info": {
      "1": 15,
      "2": 16,
      "3": 17,
      "4": 18,
      "5": 19
    }
  },
  "Probing": {
    "Ask for Explanation": {
      "1": 20,
      "2": 21,
      "3": 22,
      "4": 23,
      "5": 24
    },
    "Seek Self-Correction": {
      "1": 25,
      "2": 26,
      "3": 27,
      "4": 28,
      "5": 29
    },
    "Hypothetical Variation": {
      "1": 30,
      "2": 31,
      "3": 32,
      "4": 33,
      "5": 34
    },
    "Check Understanding/Concept": {
      "1": 35,
      "2": 36,
      "3": 37,
      "4": 38,
      "5": 39
    },
    "Encourage Comparison": {
      "1": 40,
      "2": 41,
      "3": 42,
      "4": 43,
      "5": 44
    }
  },
  "Telling": {
    "Partial Reveal (Strategy)": {
      "1": 45,
      "2": 46,
      "3": 47,
      "4": 48,
      "5": 49
    },
    "Full Reveal (Answer)": {
      "1": 50,
      "2": 51,
      "3": 52,
      "4": 53,
      "5": 54
    },
    "Corrective Explanation": {
      "1": 55,
      "2": 56,
      "3": 57,
      "4": 58,
      "5": 59
    }
  },
  "Generic": {
    "Greeting/Farewell": {
      "1": 60,
      "2": 61,
      "3": 62,
      "4": 63,
      "5": 64
    },
    "Acknowledgment/Praise": {
      "1": 65,
      "2": 66,
      "3": 67,
      "4": 68,
      "5": 69
    },
    "Summarize Progress": {
      "1": 70,
      "2": 71,
      "3": 72,
      "4": 73,
      "5": 74
    },
    "General Inquiry/Filler": {
      "1": 75,
      "2": 76,
      "3": 77,
      "4": 78,
      "5": 79
    }
  }
}

def get_next_action_id(row):
    """
    row["next_action_category"] -> e.g. "Focus"
    row["next_action_subcategory"] -> e.g. "Seek Next Step"
    row["next_action_hint_strength"] -> now 0..4 in CSV,
                                        but dictionary expects 1..5 as string
    """
    cat = row["next_action_category"]
    sub = row["next_action_subcategory"]
    # We add 1 because your CSV hint_strength is 0..4; the dictionary keys are 1..5 as strings
    hint_str = str(row["next_action_hint_strength"] + 1)
    return ACTION_MAP[cat][sub][hint_str]

# Load the CSV
df = pd.read_csv("/content/mathdial_first550.csv")

In [53]:
df

,misconception_type,listen_to_feedback,problem_progress,problem_concept,conversation_turn,correct_solution,problem_difficulty,next_action_hint_strength,next_action_category,next_action_subcategory
0,problem_interpretation_error,0,0,fractions_decimals_and_percents,1,0,2,2,Probing,Check Understanding/Concept
1,problem_interpretation_error,1,50,fractions_decimals_and_percents,2,0,2,3,Probing,Check Understanding/Concept
2,problem_interpretation_error,1,100,fractions_decimals_and_percents,3,1,2,0,Generic,Acknowledgment/Praise
3,problem_interpretation_error,0,0,basic_arithmetic,1,0,2,2,Probing,Check Understanding/Concept
4,problem_interpretation_error,1,25,basic_arithmetic,2,0,2,2,Probing,Check Understanding/Concept
...,...,...,...,...,...,...,...,...,...,...
544,problem_interpretation_error,1,100,basic_arithmetic,3,1,2,0,Generic,Acknowledgment/Praise
545,operation_error,0,0,basic_arithmetic,1,0,2,0,Generic,Greeting/Farewell
546,operation_error,0,25,basic_arithmetic,2,0,2,2,Focus,Re-direct to Sub-Problem
547,operation_error,1,75,basic_arithmetic,3,0,2,2,Focus,Seek Next Step


In [54]:
import numpy as np
# First, let's check which rows have non-finite values in next_action_hint_strength
non_finite_mask = ~np.isfinite(df["next_action_hint_strength"])
print(f"Number of non-finite values: {non_finite_mask.sum()}")
# Check which rows have non-finite values in the next_action_hint_strength column
non_finite_mask = ~np.isfinite(df["next_action_hint_strength"])

# Get the indices of rows with non-finite values
non_finite_indices = df.index[non_finite_mask].tolist()
print(f"Row indices with non-finite values: {non_finite_indices}")

Number of non-finite values: 0
Row indices with non-finite values: []


In [55]:
# Add error tracking
error_rows = []

# Process each row individually instead of using apply
for index, row in df.iterrows():
    try:
        df.loc[index, "next_action_id"] = get_next_action_id(row)
    except KeyError as e:
        error_info = {
            'index': index,
            'error': str(e),
            'category': row.get('next_action_category', 'N/A'),
            'subcategory': row.get('next_action_subcategory', 'N/A'),
            'hint_strength': row.get('next_action_hint_strength', 'N/A')
        }
        error_rows.append(error_info)
        df.loc[index, "next_action_id"] = -1  # Set a default value
        print(f"Error at row {index}: {e}")

if error_rows:
    print("\nDetailed error information:")
    for row_info in error_rows:
        print(f"Row {row_info['index']}: Category='{row_info['category']}', "
              f"Subcategory='{row_info['subcategory']}', "
              f"Hint Strength='{row_info['hint_strength']+1}', "
              f"Error: {row_info['error']}")

# Continue with the rest of your code

In [56]:
df["next_action_hint_strength"] = df["next_action_hint_strength"].astype(int)

# If needed, sort by conversation and turn
# df.sort_values(by=["conversation_id", "convo_turn"], inplace=True)
df.reset_index(drop=True, inplace=True)

# Create progress_delta
df["progress_delta"] = 0
for i in range(1, len(df)):
    if df.loc[i, "conversation_turn"] != 1:
        df.loc[i, "progress_delta"] = df.loc[i, "problem_progress"] - df.loc[i-1, "problem_progress"]

# Map each row to a unique next_action_id
df["next_action_id"] = df.apply(get_next_action_id, axis=1)

# previous_action_id: for the first turn (convo_turn=1), set 0; else, the previous row's next_action_id
df["previous_action_id"] = 0
for i in range(1, len(df)):
    if df.loc[i, "conversation_turn"] != 1:
        df.loc[i, "previous_action_id"] = df.loc[i - 1, "next_action_id"]

# Save the result
df.to_csv("/content/550_enhanced.csv", index=False)

In [57]:
# Load the data
full_df = pd.read_csv("/content/550_enhanced.csv")
full_df

,misconception_type,listen_to_feedback,problem_progress,problem_concept,conversation_turn,correct_solution,problem_difficulty,next_action_hint_strength,next_action_category,next_action_subcategory,next_action_id,progress_delta,previous_action_id
0,problem_interpretation_error,0,0,fractions_decimals_and_percents,1,0,2,2,Probing,Check Understanding/Concept,37,0,0
1,problem_interpretation_error,1,50,fractions_decimals_and_percents,2,0,2,3,Probing,Check Understanding/Concept,38,50,37
2,problem_interpretation_error,1,100,fractions_decimals_and_percents,3,1,2,0,Generic,Acknowledgment/Praise,65,50,38
3,problem_interpretation_error,0,0,basic_arithmetic,1,0,2,2,Probing,Check Understanding/Concept,37,0,0
4,problem_interpretation_error,1,25,basic_arithmetic,2,0,2,2,Probing,Check Understanding/Concept,37,25,37
...,...,...,...,...,...,...,...,...,...,...,...,...,...
544,problem_interpretation_error,1,100,basic_arithmetic,3,1,2,0,Generic,Acknowledgment/Praise,65,25,2
545,operation_error,0,0,basic_arithmetic,1,0,2,0,Generic,Greeting/Farewell,60,0,0
546,operation_error,0,25,basic_arithmetic,2,0,2,2,Focus,Re-direct to Sub-Problem,12,25,60
547,operation_error,1,75,basic_arithmetic,3,0,2,2,Focus,Seek Next Step,2,50,12
